# Validation SSD with SubT datasets

# Clone SSD Function

In [ ]:
!git clone https://github.com/ARG-NCTU/ssd.pytorch.git

Cloning into 'ssd.pytorch'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1483 (delta 4), reused 10 (delta 3), pack-reused 1463
Receiving objects: 100% (1483/1483), 177.00 MiB | 34.52 MiB/s, done.
Resolving deltas: 100% (879/879), done.


In [ ]:
import sys
sys.path.append('/content/ssd.pytorch')

# Import Libraries

In [ ]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import matplotlib.pyplot as plt
import torchvision
import random
import os.path as osp

# Download Datasets

In [ ]:
import logging
import gdown
from zipfile import ZipFile
dataset_url = 'https://drive.google.com/u/1/uc?id=1IM0LLRhGdf5IPsbtlkNFMsiGNk6quwqc&export=download'
dataset_name = 'SSD'
if not os.path.isdir(dataset_name):
    gdown.download(dataset_url, output=dataset_name + '.zip', quiet=False)
    zip1 = ZipFile(dataset_name + '.zip')
    zip1.extractall(dataset_name)
    zip1.close()


print("Finished downloading dataset.")
DATASET_ROOT = "/content/SSD/SSD-datasets"
DATA_DETECTION = SUBTDetection
cfg = SubT

Finished downloading dataset.


In [ ]:
if torch.cuda.is_available():
    if not True:
        print("WTF are u wasting your CUDA device?")
    else:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

# Define class and matching index

In [ ]:
label_idx = {'missle': 0, 'backpack': 1, 'blueline': 2, 'drill': 3, 'can': 4}
label_idx = sorted(label_idx.items(), key=lambda kv: kv[1])
label = ['background']
for i in label_idx:
    label.append(i[0])
print(label)

['background', 'missle', 'backpack', 'blueline', 'drill', 'can']


# Use pre-trained model

In [ ]:
models_url = "https://drive.google.com/uc?export=download&id=1QSYjjv-5URreZ2VvbCkZITXllYxBE6do"
models_name = "SSD.pth"
if not os.path.isfile("output/" + models_name):
    gdown.download(models_url,models_name, quiet=False)

print("Finished downloading models.")

vaild_net = build_ssd('test', 300, 6)    # initialize SSD, +1 for background
vaild_net.load_weights('/content/SSD.pth')

Downloading...
From: https://drive.google.com/uc?export=download&id=1QSYjjv-5URreZ2VvbCkZITXllYxBE6do
To: /content/SSD.pth
97.1MB [00:00, 303MB/s]


Finished downloading models.
Loading weights into state dict...
Finished!


/content/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


# Evaling

This part will initialize the ssd model and load the weight , then randomly select an image to make a prediction

In [ ]:
import xml.etree.ElementTree as ET
IMAGE_ROOT = "/content/SSD/SSD-datasets/JPEGImages"
ann_root = "/content/SSD/SSD-datasets/Annotations"
test_file = '/content/SSD/SSD-datasets/ImageSets/Main/test.txt'

In [ ]:
img_list = []
color_BBX = (255, 255, 0)
color_TEXT = (255, 0, 0)
colors = plt.cm.hsv(np.linspace(0, 1, 10)).tolist()
for line in open(osp.join(test_file)):
    img_list.append(line.strip())

accuracy = [0,0,0,0,0,0]
num = [0,0,0,0,0,0]
iou = [0,0,0,0,0,0]
m = [0,0,0,0,0,0]
print('Start Evaling...','\n')
# Read image and annotation file
for k in img_list:
  img_path = IMAGE_ROOT + '/' + k + '.jpg'
  ana_path = ann_root + '/' + k + '.xml'

  image = cv2.imread(img_path)
  rgb_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  x = cv2.resize(image, (300, 300)).astype(np.float32)
  x -= (104.0, 117.0, 123.0)
  x = x.astype(np.float32)
  x = x[:, :, ::-1].copy()
  x = torch.from_numpy(x).permute(2, 0, 1)
  xx = Variable(x.unsqueeze(0)).cuda()
  y = vaild_net(xx)
  detections = y.data
  elem = ET.parse(ana_path)
  root = elem.getroot()
  scale = torch.Tensor(rgb_image.shape[1::-1]).repeat(2)
  bboxx = [0,0]
  bboxy = [0,0]
  c = 0
  for obj in root.findall('object'):
    # Get class name
    class_name = obj.find('name').text
    for l in obj.findall('polygon'):
      c = 0
      for u in l.findall('pt'):
           # Get grown truth bounding box
           if u.find('time').text[0] == '0':
             bboxx[c] = float((u.find('x').text))
             bboxy[0] = float((u.find('y').text))
             c += 1
           if u.find('time').text[0] == 'u':
             bboxy[1] = float((u.find('y').text))
  prob = []
  objs = []
  for i in range(detections.size(1)): # detections.size(1) --> class size
             for j in range(5): # each class choose top 5 predictions
                 if detections[0, i, j, 0].cpu().numpy() > 0.5:
                     score = detections[0, i, j, 0]
                     pt = (detections[0, i, j,1:]*scale).cpu().numpy()
                     objs.append([pt[0], pt[1], pt[2]-pt[0]+1, pt[3]-pt[1]+1, i, score])
  if len(list(objs)) != 0:
    s = objs[0][4]
    pred_mask = [objs[0][0], objs[0][0]+objs[0][2], objs[0][1], objs[0][1]+objs[0][3]]
  else:
    s = 0
  d = label.index(class_name)
  gt_mask = [bboxx[0] ,bboxx[1], bboxy[0], bboxy[1]]


  num[d] += 1

  # if prediction is correct , calculate IOU
  if label[s] == class_name:
    accuracy[s] += 1
    # cal IOU
    ixmin = np.maximum(gt_mask[0], pred_mask[0])
    iymin = np.maximum(gt_mask[2], pred_mask[2])
    ixmax = np.minimum(gt_mask[1], pred_mask[1])
    iymax = np.minimum(gt_mask[3], pred_mask[3])
    iw = np.maximum(ixmax - ixmin, 0.)
    ih = np.maximum(iymax - iymin, 0.)
    inters = iw * ih
    uni = ((pred_mask[1] - pred_mask[0]) * (pred_mask[3] - pred_mask[2]) +(gt_mask[1] - gt_mask[0]) *(gt_mask[3] - gt_mask[2]) - inters)
    overlaps = float(inters / uni)
    # calculate avg IOU
    iou[s] += overlaps
for i in range(1,6):
  iou[i] = iou[i]/accuracy[i]

# show result  
for i in range(1,6):
  if accuracy[i] == 0:
    print(label[i],':')
    print('Recognition rate is ','0/',num[i],' -> 0 %')
    print('Average IOU : None', '\n')
  if accuracy[i] != 0:
    print(label[i],':')
    print('Recognition rate is ',accuracy[i],'/',num[i],' -> ',100*(accuracy[i]/num[i]), ' %')
    print('Average IOU : ',iou[i], '\n')

print('Evaling end')

Start Evaling... 

missle :
Recognition rate is  16 / 19  ->  84.21052631578947  %
Average IOU :  0.7605110857233762 

backpack :
Recognition rate is  19 / 19  ->  100.0  %
Average IOU :  0.8765026375325536 

blueline :
Recognition rate is  11 / 11  ->  100.0  %
Average IOU :  0.8936880811900902 

drill :
Recognition rate is  27 / 28  ->  96.42857142857143  %
Average IOU :  0.8287362760542979 

can :
Recognition rate is  20 / 23  ->  86.95652173913044  %
Average IOU :  0.8552684461711868 

Evaling end
